## 16. LDA Mallet 모델 만들기 > LDA보다 성능이 좋음 
지금까지는 Gensim내장 LDA알고리즘을 살펴보았습니다. 그러나 Mallet 버전은 종종 더 나은 토픽을 제공합니다. Gensim은 Gensim 자체적으로 말렛의 LDA를 구현하는 래퍼를 제공합니다. zip파일을 다운로드 하고 압축을 풀고 압축이 해제된 디렉토리에 있는 Mallet 경로를 gensim.models.wrappers.LdaMallet 에 제공하기만 하면 됩니다.(역주: Window에서는 MALLET_HOME변수에 Mallet의 home directory 경로를 설정해 주어야합니다.) 어떻게 하면 되는지 살펴봅시다.

In [ ]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = 'path/to/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))
 
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

## 17. LDA에 대한 최적의 토픽수를 찾는 방법은 무엇인가?
최적의 주제수를 찾는 접근방식은 토픽수(k)가 다른 여러 LDA모델을 작성하고 가장 높은 일관성 값을 제공하는 LDA모델을 선택하는 것입니다. 토픽 일관성의 급속한 성장의 끝을 나타내는 ‘k’를 선택하는 것은 일반적으로 의미있고 해석 가능한 토픽들을 제공합니다. 더 높은 값을 선택하면 때로는 보다 세부적인 하위주제를 제공할수도 있습니다.

여러 주제에서 동일한 키워드가 반복되는 경우 ‘k’가 너무 크다는 것일 수 있습니다.

compute_coherence_values() (아래참조)는 여러 LDA모델을 교육하고 모델및 해당 하는 일관성 점수를 제공합니다.



In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
"""
Compute c_v coherence for various number of topics
 
Parameters:
----------
dictionary : Gensim dictionary
corpus : Gensim corpus
texts : List of input texts
limit : Max num of topics
 
Returns:
-------
model_list : List of LDA topic models
coherence_values : Coherence values corresponding to the LDA model with respective number of topics
"""
coherence_values = []
model_list = []
for num_topi# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)cs in range(start, limit, step):
model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
model_list.append(model)
coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_values.append(coherencemodel.get_coherence())
 
return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

일관성 점수가 계속 증가하는 것으로 보인다면 낮아지기 전에 가장 높은 CV를 얻은 모델을 선텍하는 것이 바람직 할 수 있습니다. 이것은 바로 이 케이스에 해당합니다.

그래서 다음 단계를 위해서 20가지 토픽을 가진 모델을 선택합니다.

In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

## 18. 각 문장에서 지배적인 토픽 찾기
토픽 모델링의 실제 적용중 하나는 주어진 문서가 어떤 토픽인지 판단하는 것입니다. 이를 찾아내기 위해, 해당 문서에서 가장 높은 비율로 기여한 토픽 번호를 찾아야합니다. 아래의 format_topics_sentences()  함수는 이 정보를 알기쉽게 테이블로 집계하여 표시하여줍니다.

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
# Init output
sent_topics_df = pd.DataFrame()
 
# Get main topic in each document
for i, row in enumerate(ldamodel[corpus]):
row = sorted(row, key=lambda x: (x[1]), reverse=True)
# Get the Dominant topic, Perc Contribution and Keywords for each document
for j, (topic_num, prop_topic) in enumerate(row):
if j == 0: # => dominant topic
wp = ldamodel.show_topic(topic_num)
topic_keywords = ", ".join([word for word, prop in wp])
sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
else:
break
sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
 
# Add original text to the end of the output
contents = pd.Series(texts)
sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
return(sent_topics_df)
 
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)
 
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
 
# Show
df_dominant_topic.head(10)

## 19. 각 토픽별로 가장 대표적인 문서 찾기
때때로 토픽 키워드만으로는 토픽이 무엇인가 이해할수 없을 수도 있습니다. 따라서 토픽을 이해하는데 도움이 되도록 주어진 토픽이 가장 많이 기여한 문서를 찾아 해당 토픽을 추론하여 토픽을 추론할 수 있습니다.

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()
 
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')
 
for i, grp in sent_topics_outdf_grpd:
sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet,
grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)],
axis=0)
 
# Reset Index
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)
 
# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]
 
# Show
sent_topics_sorteddf_mallet.head()

위의 표 출력에는 실제로 토픽에 대해 각각의 20개의 행이 있습니다. 토픽번호, 키워드 및 가장 대표적인 문서가 있습니다. Perc_Contribution열은 주어진 문서에서 토픽이 차지하는 비율입니다.

## 20. 문서 전체적인 토픽 분포
마지막으로 토픽의 양과 분포를 이해하여 토픽이 얼마나 넓게 논의되었는지 판단합니다. 아래 표는 해당 정보를 제공합니다.

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
 
# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)
 
# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]
 
# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)
 
# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']
 
# Show
df_dominant_topics

## 20. 결론
우리는 토픽모델링이 할수 있는 것을 이해하는 것으로 시작을 했습니다. Gensim의 LDA사용하여 기본 토픽 모델을 작성하고 pyLDAvis를 사용하여 토픽을 시각화했습니다. 그 다음 말렛의 LDA구현을 이용했습니다. 토픽 일관성 점수를 사용하여 최적의 주제수를 찾는 방법과 최적의 모델을 선택하는 방법을 논리적으로 이해하는 방법을 보았습니다.

마지막으로 결과를 집계하고 제시하여 실용적인 통찰을 얻는 방법을 살펴보았습니다. 당신이 이 글을 재밌게 보았으면 좋겠습니다. 아래 코멘트 섹션에 의견을 남겨주시면 감사하겠습니다.
